In [1]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd 
headers = {'User Agent':'Mozilla/5.0'}

players = ['payton-pritchard', 'daniel-oturu', 'elijah-hughes', 'jalen-harris-2', 'xavier-tillman', 'tre-jones',
           'jahmius-ramsey', 'jaden-mcdaniels', 'zeke-nnaji', 'isaac-okoro', 'anthony-edwards-2', 'james-wiseman', 
           'patrick-williams-2',  'onyeka-okongwu', 'obadiah-toppin', 'jalen-smith', 'devin-vassell', 
           'tyrese-haliburton', 'kira-lewisjr', 'aaron-nesmith', 'cole-anthony', 'isaiah-stewart-2', 
           'josh-green-2','saddiq-bey', 'precious-achiuwa', 'tyrese-maxey', 'immanuel-quickley', 'udoka-azubuike', 
           'malachi-flynn', 'desmond-bane', 'tyrell-terry', 'vernon-careyjr', 'tyler-bey', 'saben-lee', 
           'robert-woodard-2', 'nick-richards-2', 'jordan-nwora', 'cj-elleby', 'nico-mannion', 'isaiah-joe', 
           'skylar-mays', 'justinian-jessup', 'cassius-winston', 'cassius-stanley', 'jay-scrubb', 'grant-riller', 
           'reggie-perry-2', 'paul-reed-5', 'sam-merrill']

player_stats = []
playerlist = []
year_list = []
age_list = []

for player in players:
    try:
        if player[-1].isdigit():
            url = f'https://www.sports-reference.com/cbb/players/{player}.html'
        else:
            url = f'https://www.sports-reference.com/cbb/players/{player}-1.html'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        header = [th.getText() for th in soup.findAll('tr', limit = 2)[0].findAll('th')]
        rows = soup.findAll('tr')
        player_stats.append([td.getText() for td in soup.find('tr', id ='players_per_game.2020')])
        table = pd.DataFrame(player_stats, columns = header) 
        for i in player_stats:
            if i[14] == '':
                i[14] = str(0.0)
        
        player = player.replace('-',' ').title()
        if player[-1].isdigit():
            player = player[:-2]
        if player[-2:] == 'jr':
            player = player.replace('jr', ' Jr')
        playerlist.append(player)
        
    except:
        continue
    try:
        player = player.replace('-', '+')
        url = f'https://www.google.com/search?q={player}+basketball+birth+date'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        age = (soup.findAll('div', class_ = 'BNeawe iBp4i AP7Wnd')[0])
        for x in age:
            x = x.text
        x = x.replace(',','').split()
        months = {
            'Janurary': '1',
            'February': '2',
            'March': '3',
            'April': '4',
            'May': '5',
            'June': '6',
            'July': '7',
            'August': '8',
            'September': '9',
            'October': '10',
            'November': '11',
            'December': '12'
        } 
        birth_date = months[x[0]]+ '-' + x[1] + '-' + x[2]
        draft_date = '7-1-2020'
        import datetime 
        birth_date_final = datetime.datetime.strptime(birth_date, '%m-%d-%Y')  
        draft_date_final = datetime.datetime.strptime(draft_date, '%m-%d-%Y')

        age_on_draft_night = draft_date_final - birth_date_final
        age = round(age_on_draft_night.days/365, 1)
        age_list.append(age)
        year = round(((1-(age/35))*2.33), 2)
        year_list.append(year)
        
    except:
        age_list.append(0)
        year_list.append(.85)

age_list[0] = 22.4
year_list[0] = .75
age_list[1] = 20.8
year_list[1] = .90
age_list[2] = 22.3
year_list[2] = .76
age_list[3] = 21.8
year_list[3] = .81
age_list[4] = 21.4
year_list[4] = .84
age_list[5] = 20.4
year_list[5] = .94
age_list[6] = 19.1
year_list[6] = 1.05
age_list[7] = 19.8
year_list[7] = 1
age_list[8] = 19.5
year_list[8] = 1.03
age_list[9] = 19.4
year_list[9] = 1.04

table.insert(0, "Name", playerlist)
table.insert(2, "Year", year_list)
table.insert(2, "Age", age_list)
table['MP'] = table['MP'].astype(float)
table['PTS'] = table['PTS'].astype(float)
table['AST'] = table['AST'].astype(float)
table['TRB'] = table['TRB'].astype(float)
table['BLK'] = table['BLK'].astype(float)
table['STL'] = table['STL'].astype(float)
table['3P'] = table['3P'].astype(float)
table['3PA'] = table['3PA'].astype(float)
table['TOV'] = table['TOV'].astype(float)
table['SOS'] = table['SOS'].astype(float)
table['PF'] = table['PF'].astype(float)
table['FT%'] = table['FT%'].astype(float)
table['3P%'] = table['3P%'].astype(float)

table["Player Grade"] = ((table['PTS']) + (table['TRB']*1.25) + (table['AST']*2) +
(table['BLK']*2) + (table['STL']*3) + (table['3P']*2) + (table['3PA']) + (table['SOS']/2)) * table['Year']

table["Player Grade"] = table["Player Grade"]*1.75
table["Player Grade"] = (round(table["Player Grade"], 1))
table["Player Grade"]= table["Player Grade"].astype(float)
                        
table = table.sort_values(by= "Player Grade", ascending=False)
table.insert(5, 'SoS', table['SOS'])
table.reset_index(drop = True, inplace=True)
import numpy as np
table.index = np.arange(1, len(table)+1)
del table['SOS']
del table['Conf']
del table['ORB']
del table['DRB']
del table['GS']
del table['FG']
del table['FGA']
del table['FT']
del table['FTA']
del table['2P']
del table['2PA']
del table['2P%']
del table['Year']
del table['TOV']
del table['PF']

table

,Name,Season,Age,School,SoS,G,MP,FG%,3P,3PA,3P%,FT%,TRB,AST,STL,BLK,PTS,,Player Grade
1,Tyrese Haliburton,2019-20,20.4,Iowa State,10.07,22,36.7,.504,2.4,5.6,0.419,0.822,5.9,6.5,2.5,0.7,15.2,,101.7
2,Kira Lewis Jr,2019-20,19.2,Alabama,8.12,31,37.6,.459,1.8,4.9,0.366,0.802,4.8,5.2,1.8,0.6,18.5,,99.3
3,Anthony Edwards,2019-20,18.9,Georgia,8.06,32,33.0,.402,2.3,7.7,0.294,0.772,5.2,2.8,1.3,0.6,19.1,,98.5
4,Aaron Nesmith,2019-20,20.7,Vanderbilt,6.41,14,35.7,.512,4.3,8.2,0.522,0.825,4.9,0.9,1.4,0.9,23.0,,94.6
5,Cole Anthony,2019-20,20.1,UNC,10.05,22,34.9,.380,2.2,6.4,0.348,0.750,5.7,4.0,1.3,0.3,18.5,,93.5
6,Cj Elleby,2019-20,20.1,Washington State,4.94,32,33.4,.396,2.3,6.8,0.339,0.823,7.8,1.9,1.8,0.8,18.4,,91.5
7,Isaiah Joe,2019-20,21.0,Arkansas,7.27,26,36.1,.367,3.6,10.6,0.342,0.890,4.1,1.7,1.4,0.3,16.9,,84.1
8,Desmond Bane,2019-20,22.0,TCU,10.09,32,36.0,.452,2.9,6.5,0.442,0.789,6.3,3.9,1.5,0.5,16.6,,83.9
9,Tre Jones,2019-20,20.4,Duke,8.00,29,35.4,.423,1.3,3.7,0.361,0.771,4.2,6.4,1.8,0.3,16.2,,83.2
10,Paul Reed,2019-20,21.1,DePaul,8.95,29,31.7,.516,0.6,1.8,0.308,0.738,10.7,1.6,1.9,2.6,15.1,,81.5
